# Integrating an ONNX Model

This notebook shows how to create an integration using a scikit-learn model that is converted to ONNX

In [ ]:
# Import required packages

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
import onnxruntime as ort
import numpy as np
import skl2onnx

import aisquared

In [ ]:
# Load the data

data = np.array(fetch_20newsgroups()['data'])
target = fetch_20newsgroups()['target']
target_names = fetch_20newsgroups()['target_names']

true_targets = []
for i in range(target.shape[0]):
    true_targets.append(target_names[target[i]])
true_targets = np.array(true_targets)

In [ ]:
# Build the model
pipeline = Pipeline(
    [
        ('vectorizer', TfidfVectorizer(stop_words = 'english', max_df = 0.8, min_df = 5)),
        ('clf', DecisionTreeClassifier(max_depth = 10))
    ]
)

# Fit the model
pipeline.fit(data, true_targets)
print(pipeline.predict(data))

In [ ]:
# Convert the model to ONNX
onnx_pipeline = skl2onnx.to_onnx(
    pipeline,
    data,
    options = {'zipmap' : False}
)

with open('classifier.onnx', 'wb') as f:
    f.write(onnx_pipeline.SerializeToString())

In [ ]:
# Run the onnx model
sess = ort.InferenceSession('classifier.onnx')
print(sess.run(None, {'X' : data}))

In [ ]:
# Create the .air file

harvester = aisquared.config.harvesting.ChatbotHarvester('ONNX Text Classifier')
preprocesser = None
analytic = aisquared.config.analytic.OnnxModel('classifier.onnx', [1], output_key = 'label', return_key = 'predicted_class')
postprocesser = None
renderer = aisquared.config.rendering.ChatRendering('predicted_class', responder_name = 'ONNX Classifier')

config = aisquared.config.ModelConfiguration(
    'Example ONNX Text Classifier',
    harvester,
    preprocesser,
    analytic,
    postprocesser,
    renderer
)
config.compile()